<a href="https://colab.research.google.com/github/woodRock/fishy-business/blob/main/code/gp/notebooks/Wrapper_Based_Multi_tree_Genetic_Program.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi-tree Genetic Program 

## Kicked for inactivity?

To stop a colab notebook from disconnecting, open up the console with CTRL + SHIFT + I, and copy and execute the following code.

```javascript
function ConnectButton(){
    console.log("Connect pushed"); 
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() 
}
setInterval(ConnectButton,60000);
```

This tricks the browser into thinking the user is active.

In [ ]:
!pip install deap
!apt install libgraphviz-dev
!pip install pygraphviz
!pip install skfeature-chappers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libgraphviz-dev is already the newest version (2.40.1-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
data_path_gdrive = '/content/drive/MyDrive/AI/Data'
dataset = 'Fish' #@param ["Fish", "Part"]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Data

In [ ]:
"""
Data - data.py
==============

This is the data module. It contains the functions for loading, preparing, normalizing and encoding the data.
"""

import numpy as np 
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
import scipy.io
import os

def encode_labels(y, y_test=None):
    """
    Convert text labels to numbers.

    Args:
        y: The labels.
        y_test: The test labels. Defaults to None.
    """
    le = preprocessing.LabelEncoder()
    y = le.fit_transform(y)
    if y_test is not None:
        y_test = le.transform(y_test)
    return y, y_test, le


def load(filename, folder=''):
    """
    Load the data from the mat file.

    Args:
        filename: The name of the mat file.
        folder: The folder where the mat file is located.
    """
    path = os.path.join(folder, filename)
    mat = scipy.io.loadmat(path)
    return mat


def prepare(mat):
    """
    Load the data from matlab format into memory. 

    Args:
        mat: The data in matlab format.
    """
    X = mat['X']   
    X = X.astype(float)
    y = mat['Y']    
    y = y[:, 0]
    return X,y


def normalize(X_train, X_test):
    """
    Normalize the input features within range [0,1].

    Args:
        X_train: The training data.
        X_test: The test data.
    """
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaler = scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    return X_train, X_test

file = load(f'{dataset}.mat', folder=data_path_gdrive)
X,y = prepare(file)
X,_ = normalize(X,X)
y, _, le = encode_labels(y)
labels = le.inverse_transform(np.unique(y))
classes, class_counts = np.unique(y, return_counts=True)
n_features = X.shape[1]
n_instances = X.shape[0]
n_classes = len(classes)
class_ratios = np.array(class_counts) / n_instances

print(f"Class Counts: {class_counts}, Class Ratios: {class_ratios}")
print(f"Number of features: {n_features}\nNumber of instances: {n_instances}\nNumber of classes {n_classes}.")

Class Counts: [68 26 31 29], Class Ratios: [0.44155844 0.16883117 0.2012987  0.18831169]
Number of features: 4800
Number of instances: 154
Number of classes 4.


## Activation Function

## Operators

In [ ]:
import math
import copy
import random
import operator
from re import I
from operator import attrgetter
from functools import wraps, partial

import numpy as np

from deap import algorithms
from deap.algorithms import varAnd
from deap import base, creator, tools, gp
from deap.gp import PrimitiveTree, Primitive, Terminal

from sklearn.metrics import balanced_accuracy_score

pset = gp.PrimitiveSet("MAIN", n_features)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(operator.neg, 1)
# pset.addEphemeralConstant("rand101", lambda: random.randint(-1,1))

## Fitness Function

In [ ]:
toolbox = base.Toolbox()

minimized = False
if minimized: 
    weight = -1.0 
else: 
    weight = 1.0

weights = (weight,) 

if minimized: 
    creator.create("FitnessMin", base.Fitness, weights=weights)
    creator.create("Individual", list, fitness=creator.FitnessMin)
else:
    creator.create("FitnessMax", base.Fitness, weights=weights)
    creator.create("Individual", list, fitness=creator.FitnessMax)

def quick_evaluate(expr: PrimitiveTree, pset, data, prefix='ARG'):
    """ Quick evaluate offers a 500% speedup for the evluation of GP trees.

    The default implementation of gp.compile provided by the DEAP library is 
    horrendously inefficient. (Zhang 2022) has shared his code which leads to a 
    5x speedup in the compilation and evaluation of GP trees when compared to the
    standard library approach.

    For multi-tree GP, this speedup factor is invaluable! As each individual conists
    of m trees. For the fish dataset we have 4 classes, each with 3 constructed features,
    which corresponds to 4 classes x 3 features = 12 trees for each individual. 
    12 trees x 500% speedup = 6,000% overall speedup, or 60 times faster. 
    The 500% speedup is fundamental, for efficient evaluation of multi-tree GP. 

    Args:
        expr (PrimitiveTree): The uncompiled (gp.PrimitiveTree) GP tree.
        pset: The primitive set.
        data: The dataset to evaluate the GP tree for.
        prefix: Prefix for variable arguments. Defaults to ARG.

    Returns: 
        The (array-like) result of the GP tree evaluate on the dataset .
    """
    result = None
    stack = []
    for node in expr:
        stack.append((node, []))
        while len(stack[-1][1]) == stack[-1][0].arity:
            prim, args = stack.pop()
            if isinstance(prim, Primitive):
                result = pset.context[prim.name](*args)
            elif isinstance(prim, Terminal):
                if prefix in prim.name:
                    result = data[:, int(prim.name.replace(prefix, ''))]
                else:
                    result = prim.value
            else:
                raise Exception
            if len(stack) == 0:
                break # If stack is empty, all nodes should have been seen
            stack[-1][1].append(result)
    return result

def compileMultiTree(expr, pset):
    """Compile the expression represented by a list of trees. 

    A variation of the gp.compileADF method, that handles Multi-tree GP. 

    Args: 
        expr: Expression to compile. It can either be a PrimitiveTree,
                 a string of Python code or any object that when
                 converted into string produced a valid Python code
                 expression.
        pset: Primitive Set

    Returns: 
        A set of functions that correspond for each tree in the Multi-tree. 
    """
    funcs = []
    gp_tree = None
    func = None

    for subexpr in expr:
        gp_tree = gp.PrimitiveTree(subexpr)
        # 5x speedup by manually parsing GP tree (Zhang 2022) https://mail.google.com/mail/u/0/#inbox/FMfcgzGqQmQthcqPCCNmstgLZlKGXvbc
        func = quick_evaluate(gp_tree, pset, X, prefix='ARG')
        funcs.append(func)

    # Hengzhe's method returns the features in the wrong rotation for multi-tree
    features = np.array(funcs).T 
    return features

# MCIFC constructs 8 feautres for a (c=4) multi-class classification problem (Tran 2019).
# c - number of classes, r - construction ratio, m - total number of constructed features.
# m = r * c = 2 ratio * 4 classes = 8 features

r = 3
c = n_classes 
m = r * c 

toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.expr, n=m)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", compileMultiTree)

## Infinite speedup!?

For multi-tree GP, this speedup factor is invaluable! As each individual conists of m trees. For the fish dataset we have 4 classes, each with 3 constructed features, which corresponds to 4 classes x 3 features = 12 trees for each individual. 12 trees x 500% speedup = 6,000% overall speedup, or 60 times faster. The 500% speedup is fundamental, for efficient evaluation of multi-tree GP.

The evaluation below shows my calculations may still be wrong. And perhaps, it is even faster:

In [ ]:
first = toolbox.population(n=1)[0]

subtree = first[0]
gp_tree = gp.PrimitiveTree(subtree)

%timeit gp.compile(gp_tree, pset)
%timeit quick_evaluate(gp_tree, pset, X)

7.49 ms ± 59.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
4.03 µs ± 240 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [ ]:
from sklearn.svm import LinearSVC as svm
from sklearn.model_selection import StratifiedKFold

def wrapper_classification_accuracy(X, k=10, verbose=False):
    """ Evaluate balanced classification accuracy over stratified k-fold cross validation. 

    This method is our fitness measure for an individual. We measure each individual
    based on its balanced classification accuracy using 10-fold cross-validation on
    the training set.

    If verbose, we evaluate performance on the test set as well, and print the results
    to the standard output. By default, only the train set is evaluated, which 
    corresponds to a 2x speedup for training, when compared to the verbose method.
    
    Args:
        X: entire dataset, train and test. 
        k: Number of folds, for cross validation. Defaults to 10.
        verbose: If true, prints stuff. Defaults to false.

    Returns:
        Average balanced classification accuracy with 10-fold CV on training set.
    """ 
    train_accs = []
    test_accs = [] 
    skf = StratifiedKFold(n_splits=10)
    
    for train_idx, test_idx in skf.split(X,y):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        # Convergence errors for the fish part dataset.
        # Need to use a different SVM hyperparameters for this dataset.
        # model = svm(penalty='l2', max_iter=10_000)
        model = svm()
        model.fit(X_train, y_train)
        y_predict = model.predict(X_train)
        train_acc = balanced_accuracy_score(y_train, y_predict)
        train_accs.append(train_acc)

        # Only evaluate test set if verbose!
        # Results in 2x speedup for training.
        if verbose:
            y_predict = model.predict(X_test)
            test_acc = balanced_accuracy_score(y_test, y_predict)
            test_accs.append(test_acc)
            print(f"Train accuracy: {train_acc}, Test accuracy: {test_acc}")

    avg_train_acc = np.mean(train_accs) 
    
    if verbose:
        # Must be here, to avoid numpy warnings!
        avg_test_acc = np.mean(test_accs)
        print(f"Average train accuracy: {avg_train_acc}, Average test accuracy: {avg_test_acc}")

    return avg_train_acc

wrapper_classification_accuracy(X, verbose=True)

/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Train accuracy: 1.0, Test accuracy: 1.0


/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Train accuracy: 1.0, Test accuracy: 0.9166666666666666


/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


...

Train accuracy: 1.0, Test accuracy: 1.0


/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Train accuracy: 1.0, Test accuracy: 1.0
Train accuracy: 1.0, Test accuracy: 1.0
Average train accuracy: 1.0, Average test accuracy: 0.9845238095238095


/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


1.0

In [ ]:
#  DEBUG : REMOVE THIS !!!
# import warnings
# warnings.filterwarnings('ignore') 

def xmate(ind1, ind2):
    """ Reproduction operator for multi-tree GP, where trees are represented as a list.

    Crossover happens to a subtree that is selected at random. 
    Crossover operations are limited to parents from the same tree.

    FIXME: Have to compile the trees (manually), which is frustrating. 
    
    Args: 
        ind1 (Individual): The first parent. 
        ind2 (Individual): The second parent 

    Returns:
        ind1, ind2 (Individual, Individual): The children from the parents reproduction. 
    """
    n = range(len(ind1))
    selected_tree_idx = random.choice(n)
    for tree_idx in n:
        g1, g2 = gp.PrimitiveTree(ind1[tree_idx]), gp.PrimitiveTree(ind2[tree_idx])
        if tree_idx == selected_tree_idx:
            ind1[tree_idx], ind2[tree_idx] = gp.cxOnePoint(g1, g2)
        else: 
            ind1[tree_idx], ind2[tree_idx] = g1, g2
    return ind1, ind2


def xmut(ind, expr):
    """ Mutation operator for multi-tree GP, where trees are represented as a list. 

    Mutation happens to a tree selected at random, when an individual is selected for crossover. 

    FIXME: Have to compile the trees (manually), which is frustrating. 

    Args: 
        ind: The individual, a list of GP trees. 
    """
    n = range(len(ind))
    selected_tree_idx = random.choice(n)
    for tree_idx in n:
        g1 = gp.PrimitiveTree(ind[tree_idx])
        if tree_idx == selected_tree_idx:
            indx = gp.mutUniform(g1, expr, pset)
            ind[tree_idx] = indx[0]
        else:
            ind[tree_idx] = g1
    return ind,


def evaluate_classification(individual, alpha = 0.9, verbose=False):
    """ 
    Evalautes the fitness of an individual for multi-tree GP multi-class classification.

    We maxmimize the fitness when we evaluate the accuracy + regularization term. 

    Args:  
        individual (Individual): A candidate solution to be evaluated. 
        alpha (float): A parameter that balances the accuracy and regularization term. Defaults to 0.98.

    Returns: 
        accuracy (tuple): The fitness of the individual.
    """  
    features = toolbox.compile(expr=individual, pset=pset)
    fitness = wrapper_classification_accuracy(features, verbose=verbose)
    return fitness,


toolbox.register('evaluate', evaluate_classification)
toolbox.register("select", tools.selTournament, tournsize=7)
toolbox.register("mate", xmate)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", xmut, expr=toolbox.expr_mut)


def staticLimit(key, max_value):
    """
    A variation of gp.staticLimit that works for Multi-tree representation. 
    This works for our altered xmut and xmate genetic operators for mutli-tree GP. 
    If tree depth limit is exceeded, the genetic operator is reverted. 

    When an invalid (over the limit) child is generated,
    it is simply replaced by one of its parents, randomly selected.
    
    Args:
        key: The function to use in order the get the wanted value. For
             instance, on a GP tree, ``operator.attrgetter('height')`` may
             be used to set a depth limit, and ``len`` to set a size limit.
        max_value: The maximum value allowed for the given measurement. 
             Defaults to 17, the suggested value in (Koza 1992)
    
    Returns: 
        A decorator that can be applied to a GP operator using \
        :func:`~deap.base.Toolbox.decorate`

    References:
        1. Koza, J. R. G. P. (1992). On the programming of computers by means 
            of natural selection. Genetic programming.
    """

    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            keep_inds = [[copy.deepcopy(tree) for tree in ind] for ind in args]
            new_inds = list(func(*args, **kwargs))
            for ind_idx, ind in enumerate(new_inds):
                for tree_idx, tree in enumerate(ind):
                    if key(tree) > max_value:
                        random_parent = random.choice(keep_inds)
                        new_inds[ind_idx][tree_idx] = random_parent[tree_idx]
            return new_inds
        return wrapper
    return decorator

# See https://groups.google.com/g/deap-users/c/pWzR_q7mKJ0
toolbox.decorate("mate", staticLimit(key=operator.attrgetter("height"), max_value=8))
toolbox.decorate("mutate", staticLimit(key=operator.attrgetter("height"), max_value=8))


def SimpleGPWithElitism(population, toolbox, cxpb, mutpb, ngen, stats=None,
             halloffame=None, verbose=__debug__):
    """
    Elitism for Multi-Tree GP for Multi-Class classification. 
    A variation of the eaSimple method from the DEAP library that supports 

    Elitism ensures the best individuals (the elite) from each generation are 
    carried onto the next without alteration. This ensures the quality of the 
    best solution monotonically increases over time. 
    """
    logbook = tools.Logbook()
    logbook.header = ['gen', 'nevals'] + (stats.fields if stats else [])

    invalid_ind = [ind for ind in population if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    if halloffame is None:
        raise ValueError("halloffame parameter must not be empty!")

    halloffame.update(population)
    hof_size = len(halloffame.items) if halloffame.items else 0

    record = stats.compile(population) if stats else {}
    logbook.record(gen=0, nevals=len(invalid_ind), **record)
    
    if verbose:
        print(logbook.stream)

    for gen in range(1, ngen + 1):
        offspring = toolbox.select(population, len(population) - hof_size)
        offspring = algorithms.varAnd(offspring, toolbox, cxpb, mutpb)
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
        
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        offspring.extend(halloffame.items)
        halloffame.update(offspring)
        population[:] = offspring
        
        record = stats.compile(population) if stats else {}
        logbook.record(gen=gen, nevals=len(invalid_ind), **record)
        
        if verbose:
            print(logbook.stream)

    return population, logbook


def train(generations=100, population=100, elitism=0.1, crossover_rate=0.5, mutation_rate=0.1):
    """
    This is a Multi-tree GP with Elitism for Multi-class classification. 

    Args:
        generations: The number of generations to evolve the populaiton for. 
        elitism: The ratio of elites to be kept between generations. 
        crossover_rate: The probability of a crossover between two individuals. 
        mutation_rate: The probability of a random mutation within an individual.  

    Returns:
        pop: The final population the algorithm has evolved. 
        log: The logbook which can record important statistics. 
        hof: The hall of fame contains the best individual solutions.
    """
    random.seed(420)
    pop = toolbox.population(n=population)
    
    mu = round(elitism * population)
    if elitism > 0:
        # See https://www.programcreek.com/python/example/107757/deap.tools.HallOfFame
        hof = tools.HallOfFame(mu)
    else:
        hof = None
    
    stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
    length = lambda a: np.max(list(map(len, a)))
    stats_size = tools.Statistics(length)

    mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
    mstats.register("avg", np.mean)
    mstats.register("std", np.std)
    mstats.register("min", np.min)
    mstats.register("max", np.max)

    pop, log = SimpleGPWithElitism(pop, toolbox, crossover_rate, mutation_rate, 
                                   generations, stats=mstats, halloffame=hof, 
                                   verbose=True)
    return pop, log, hof


"""
DeJong (1975), p=50-100, m=0.001, c=0.6 
Grefenstette (1986), p=30, m=0.01, c=0.95
Schaffer et al., (1989), p=20-30, m=0.005-0.01, c=0.75-0.95

References:
    1. Patil, V. P., & Pawar, D. D. (2015). The optimal crossover or mutation 
    rates in genetic algorithm: a review. International Journal of Applied 
    Engineering and Technology, 5(3), 38-41.
"""

beta = 1
population = n_features * beta
generations = 300
elitism = 0.1
crossover_rate = 0.8
mutation_rate = 0.2

assert crossover_rate + mutation_rate == 1, "Crossover and mutation sums to 1 (to please the Gods!)"

pop, log, hof = train(generations, population, elitism, crossover_rate, mutation_rate)

   	      	                                fitness                                 	                      size                     
   	      	------------------------------------------------------------------------	-----------------------------------------------
gen	nevals	avg     	gen	max     	min     	nevals	std     	avg    	gen	max	min	nevals	std    
0  	4800  	0.582979	0  	0.803414	0.326464	4800  	0.069835	6.51375	0  	7  	3  	4800  	1.00614
1  	3617  	0.670295	1  	0.837001	0.471786	3617  	0.0476104	6.61708	1  	13 	3  	3617  	1.10455
2  	3589  	0.72362 	2  	0.858673	0.48458 	3589  	0.0390394	6.79396	2  	13 	3  	3589  	1.15029
3  	3703  	0.760041	3  	0.874104	0.484029	3703  	0.0346598	6.95833	3  	13 	3  	3703  	1.14707
4  	3628  	0.78662 	4  	0.882083	0.601666	3628  	0.0318957	7.14625	4  	15 	3  	3628  	1.19263
5  	3584  	0.806816	5  	0.890737	0.621227	3584  	0.0310009	7.37812	5  	17 	3  	3584  	1.38765
6  	3585  	0.823537	6  	0.915481	0.646436	3585  	0.0305809	7.40646	6  	13 	3  	3

/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warning

23 	3635  	0.958548	23 	0.987372	0.811331	3635  	0.0218779	8.44937	23 	18 	5  	3635  	1.37899


/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warning

24 	3595  	0.959464	24 	0.987372	0.79556 	3595  	0.0225048	8.81854	24 	19 	6  	3595  	1.72355


...

298	3621  	0.986307	298	1       	0.804629	3621  	0.0252245	12.4675	298	31 	6  	3621  	3.8584 


/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warning

299	3560  	0.985628	299	1       	0.808603	3560  	0.0265355	12.4208	299	27 	7  	3560  	3.79885


/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warning

300	3626  	0.985881	300	1       	0.781509	3626  	0.0262333	12.4848	300	27 	7  	3626  	3.8142 


In [ ]:
evaluate_classification(hof[0], verbose=True)

Train accuracy: 1.0, Test accuracy: 1.0
Train accuracy: 1.0, Test accuracy: 0.9642857142857143
Train accuracy: 1.0, Test accuracy: 1.0
Train accuracy: 1.0, Test accuracy: 1.0
Train accuracy: 1.0, Test accuracy: 1.0
Train accuracy: 1.0, Test accuracy: 1.0
Train accuracy: 1.0, Test accuracy: 1.0
Train accuracy: 1.0, Test accuracy: 1.0
Train accuracy: 1.0, Test accuracy: 1.0
Train accuracy: 1.0, Test accuracy: 1.0
Average train accuracy: 1.0, Average test accuracy: 0.9964285714285716


(1.0,)

## Visualization

In [ ]:
from deap import base, creator, gp
import pygraphviz as pgv

multi_tree = hof[0]
for t_idx,tree in enumerate(multi_tree): 
    nodes, edges, labels = gp.graph(tree)

    g = pgv.AGraph()
    g.add_nodes_from(nodes)
    g.add_edges_from(edges)
    g.layout(prog="dot")

    for i in nodes:
        n = g.get_node(i)
        n.attr["label"] = labels[i]

    g.draw(f"tree-{t_idx}.pdf")

## Changelog 

| Date | Title | Description | Update | 
| --- | --- | --- | ---- | 
| 2022-08-21 17:30 | Multi-Objective - Onehot Encoding | Change to multi-objective problem, one-vs-all with a tree classifier for each class.<br> Y labels are encoded in onehot encodings, error is absolute difference between $|\hat{y} - y|$| 
| 2022-08-22 20:44 | Non-linearity |  Introduce $ round . sigmoid $ to evaluate_classification() method.<br>Previously, we push each tree to predict either a 0 or 1 value with the onehot encoding representation.<br>Now, the non-linearity will map any negative value to a negative class 0, and any positive value to positive class 1.|
| 2022-08-22 21:06 | ~~Genetic operators for tree with worst fitness~~ | Only apply the genetic operators, crossover and mutation, to the tree with the worst fitness.<br> This guarantees monotonic improvement for the Multi-tree between generations, the best performing tree remain unaltered.| (Update) This was very slow, and inefficient,<br> basically turned the GP into a single objective,<br>that balances multi-objective fitness functions. | 
| 2022-08-22 21:15 | Halloffame Equality Operator | Numpy equality function (operators.eq) between two arrays returns the equality element wise,<br>which raises an exception in the if similar() check of the hall of fame. <br> Using a different equality function like numpy.array_equal or numpy.allclose solve this issue.| 
| 2022-08-22 23:22 | Elitism as aggregate best tree | Perform elitsim by constructing the best tree, as the tree with best fitness from each clas.<br>The goal is to have monotonous improvement across the multiple objective functions.| 
| 2022-08-22 23:32 | Update fitness for elite | The elitism was not working as intended, as the multi-objectives didn't appear to increase monotnously.<br> This was because the aggregate fitness was not being assigned to the best individual after it was created.<br>Therefore the best invidiual was not passed on to the next generation. | 
| 2022-08-22 02:28 | staticLimit max height | Rewrite the gp.staticLimit decorator function to handle the Multi-tree representation.<br>Note: size and depth are different values!<br>depth is the maximum length of a root-to-leaf traversal,<br>size is the total number of nodes.| 
| 2022-08-24 9:37 | Evaluate Mutli-tree train accuracy | Take the classification accuracy as the argmax for the aggregate multitree.<br> 74% training accuracy, which is not ideal, but this shall improve with time.| 
| 2022-08-25 13:30 | Single-objective fitness | Change the fitness function to a single objective fitness function.<br>This forces the multi-tree GP to find the best tree subset for one-vs-rest classification performance.| 
| 2022-08-25 20:01 | Fitness = Balanced accuracy + distance measure | Implement the fitness function for MCIFC, but for multi-class classification from (Tran 2019) |
| 2022-08-26 21:27 | Sklearn Balanced Accuracy | Changed to the balanced accuracy metric from sklearn.<br>This is much easier to use for now, probably faster than the previous method as well. | 
| 2022-09-05 17:00 | Reject invalid predictions | Change the fitness function to reject invalid predictioctions outright -<br>e.g. multi-label or zero-label predictions<br>- when computing the balanced accuracy for the fitness function. |
| 2022-09-13 19:00 | Mutation + Crossover = 100% | Ensure the mutation and crossover rate sum to 100%,<br>not necessary with deap, but good to avoid conference questions |
| 2022-09-13 21:00 | Feature Construction | Changed to Wrapper-based Feature Construction with Multi-tree GP.|
| 2022-09-13 21:34 | $m = r \times c$ | Add more trees, following example from (Tran 2019).<br> With 8 trees for a multi-class classification<br> $m = r \times c = 8$ trees, where number of classes $c = 4$, and reconstruction ratio $r = 2$/ | 
| 2022-09-30 19:49 | Quick Evalaute | Manually parse the GP trees, 5x speedup for DEAP in Python (Zhang 2022). | 
| 2022-10-13 6:02 | Ignore timestamps after 4500 | Ignore timestamps after 4500 did not improve accuracy for SVM classifier.<br>So the bizzare pattern that occurs on GC-MS image there has important information.<br> Should investigate this further, perhaps ask Daniel as he is a domain expert.|
| 2022-10-13 22:16 | Cross validation | Evaluate the mean balanced classification accuracy over stratified k-fold cross validation.| 
| 2023-01-13 20:58 | 2x speedup | Only evaluate test set for verbose alternative of the evaluate_classification method.<br> This results in a 2x speedup in the efficiency of the training regime.|